[View in Colaboratory](https://colab.research.google.com/github/monstrim/tf-accel-recog/blob/master/TF_accel_recog.ipynb)

# TF-accel-recog

### Goal
* To label a time series of 6 channels of motion capturing sensors (XYZ accel + gyro) according to activity performed (walking, sitting, steps up, etc). The model we're building here should be then able to learn a similar task, but involving skate tricks rather than walking/sitting.

### Dataset
* Description: http://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones
* Download link: http://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip

## Framework setup

In [0]:
!pip install tensorflow

In [0]:
import os
from zipfile import ZipFile
from urllib import request

import tensorflow as tf
import tensorflow.contrib.eager as tfe #hopefuly this won't be needed for long

tf.enable_eager_execution()

In [0]:
dataset_url = (
    'http://archive.ics.uci.edu/ml/'
    + 'machine-learning-databases/'
    + '00240/UCI%20HAR%20Dataset.zip'
)
filename = 'HAR_dataset.zip'

if not os.path.exists(filename):
    request.urlretrieve(dataset_url, filename)

with ZipFile(filename) as archive:
    files = [file for file in archive.namelist() 
             if not file.startswith('__MACOS')]
    archive.extractall(path='data', members=files)

## Getting acquainted with the data

Let's begin by checking the files we downloaded.

In [0]:
[os.path.join(dp, f) for dp, dn, fn in os.walk('data') for f in fn]

It seems from the README.txt (which we get an error when we try to open here, but _trust me I checked_), that the X_train files contain only the preprocessed values (_a whole 500+ columns of it!_), and those aren't interesting for our needs. We want the raw data, and that's in the Inertial Signals folder. Thing is, it's one file per channel, and it's batched/windowed, so we need to look at the files and see how we can tease out the raw data as a time series, one line per sample, one column per channel.

In [0]:
samplefile = (
    'data/UCI HAR Dataset/train/'
    + 'Inertial Signals/body_gyro_x_train.txt'
)

with open(samplefile) as file:
    for i in range(10):
        line = file.readline().rstrip().split()
        print(i)
        print(line[0:64])
        print(line[64:])

OK. Each line has 128 samples, at 50Hz, with 50% overlap, so the first 64 samples on each line are the same as the last 64 on the previous one. To get one long sample, we need to select the first 64 samples on each line, then concatenate lines (checking the subject_train.txt file to see if we're still in the same run?). That will become a time series for a single channel, at 50Hz. We then need to concatenate the result for different files to get a single 6-channel time series.

However, we must remember the original preprocessing (and the labeling) was performed on the 128-sample windows (64 if we disregard the overlap), so each line on the train and test files will correspond to 64 lines in our raw time series.

### Dataset pipeline test

In [0]:
raw_train_dir = 'data/UCI HAR Dataset/train/Inertial Signals/'
raw_train_files = [
    os.path.join(raw_train_dir, x)
    for x in [
        'total_acc_x_train.txt',
        'total_acc_y_train.txt',
        'total_acc_z_train.txt',
        'body_gyro_x_train.txt',
        'body_gyro_y_train.txt',
        'body_gyro_z_train.txt'
    ]
]

raw_train_files

In [0]:
dataset = (
    tf.data.Dataset
    .from_tensor_slices(raw_train_files)
    .interleave(
        lambda x: (
            #tf.data.Dataset.from_tensors(x).repeat(10)
            tf.data.TextLineDataset(x)
        ), 6
    )
    #.map(lambda x: tf.decode_csv(x, [[0]], field_delim=' '))
    #.batch(6)
    .take(10)
)
for item in tfe.Iterator(dataset):
    print(item)

WHY THE FUCK DOESN'T THIS ASSHOLE SEPARATE VALUES WITH A FUCKING COMMA??? Who uses fucking fixed width columns?? Python's string split function works just fine, but we're supposed to apply only tf functions inside a dataset map, plus split returns a list not a Tensor. SERIOUSLY, WTF.

We've tried:
* tf.decode_raw: will return one uint8 per character. We'd have to group them in 16-long batches, convert _back_ to characters, concatenate, then parse as a float. And I don't think there's a function just for parsing text values.
* tf.split_string: it needs a different shape? Can't figure this shit out, the error messages are (surprise?) _not fucking helpful_.
* tf.expand_dims + tf.split_string: Let's try adding the dimension it needs then? We can't because... hell if I know. The only thing the error message tells me it it's got something to do with being run in eager mode.

I've no idea what to do with this shit. My best guess is, abandon the Tensorflow dataset pipeline, make a preprocessing step in numpy or some shit, save the preprocessed file back to disk and work on it instead.

God fucking damn it.

## Data Preprocessing

Made peace with the fact we're gonna have to do this as an actual pre-step and save a processed file, rather than do it live in the Dataset pipeline.

### Goal
To preprocess 6 files into a single one. Each of the raw files has 128 samples per line, in fixed-width format. We need to get the first 64 of these, then transpose it, then concatenate all the lines as one long vector. That will be one column of the final file, and we need 6 of those.

Coffee machine hasn't been refilled in 4 hours tho, so I AIN'T GONNA DO THIS SHIT NOW.